In [ ]:
# Get Insight data by only replacing the *owner and project name *with appropriate *relative path *of the cloned repository from github

In [ ]:
import pandas as pd
from typing import Optional
from github import Github
from github.Repository import Repository
import os
from subprocess import check_output
import subprocess
import time

In [ ]:
owner = "facebook"
project = "react"

In [ ]:
print("Current directory ",os.getcwd())

In [ ]:
dir = "C:/Users/HASAN/Desktop/react_vue_insight_analysis/"+ project
isdir = os.path.isdir(dir)
if isdir:
    print('Repository Already cloned in this directory')
else:
    print("Cloning "+ project + " repository...")
    clone_command = "git clone https://github.com/"+ owner + "/" + project + ".git"
    tmp = check_output(clone_command, shell=True).decode()
    print(tmp)
    print("done")

In [ ]:
os.chdir(dir)
print("Current directory ",os.getcwd())

In [ ]:
labels = ['Base', 'head', 'files_changed', 'addition', 'deletion', 'Changed_lines', 'total_commit', 'total_contributor']
labels_for_author_commit = ['Base', 'head', 'Author_name', 'commit_count']

In [ ]:
csv_results = []
csv_results_for_author_commit = []
class GithubStatisticsGeneration(object):

    def __init__(self, user_name: Optional[str] = None, password: Optional[str] = None,
                 access_token: Optional[str] = None):
        self._user_name: str = user_name
        self._password: str = password
        self._access_token: str = access_token

    def get_tags(self, repository: str):
        _cred: Github = self._authenticate_to_github(self._access_token, self._password, self._user_name)
        _repo: Repository = _cred.get_repo(repository)
        re = _repo.get_releases()

        _tags = list(_repo.get_releases())
        _tags = sorted(_tags, key=lambda x: x.created_at, reverse = True)
        print(len(_tags))       
#         print(_tags[0].published_at)
        for i in range(len(_tags)- 1):
            head = _tags[i].tag_name
            base = _tags[i+1].tag_name
            print(base, head)
#             print(_tags[i].created_at)
            try:
                get_addition_deletion_command = "git diff "+ base + "..." + head + " --shortstat"
                get_number_of_commit_command = "git rev-list --count " + base + ".." + head
                get_number_of_contributor_command = "git shortlog " + base + ".." + head + ' -sn | find /c /v ""'

                print(get_addition_deletion_command)
                addition_deletion = check_output(get_addition_deletion_command, shell=True).decode()
                addition_deletion_list = addition_deletion.split(" ")
                print(addition_deletion_list, len(addition_deletion_list))
    #             print(addition_deletion_list[1], addition_deletion_list[4], addition_deletion_list[6])

                print(get_number_of_commit_command)
                commit_no = check_output(get_number_of_commit_command, shell=True).decode()
                print("commit_no: ", commit_no)

                print(get_number_of_contributor_command)
                contributor_no = check_output(get_number_of_contributor_command, shell=True).decode()
                print("contributor_no: ", contributor_no)

                total_change = int(addition_deletion_list[4])+int(addition_deletion_list[6])
                
                # Get Author individual contribution information
                get_authors_individual_commit_count_command = "git shortlog " + base + ".." + head + ' -sn'
                print(get_authors_individual_commit_count_command)
                authors_individual_commit = check_output(get_authors_individual_commit_count_command, shell=True).decode()
                authors_individual_commit_list = authors_individual_commit.splitlines()
                
                for authors_commit in authors_individual_commit_list:
                #     print(authors_commit)
                    author_commit = authors_commit.split('\t')
                    authors_commit_count = int( author_commit[0].strip() )
                    author_name = author_commit[1]
                #     print(type(author_commit), author_commit, len(author_commit))
                    print("authors_commit_count ", authors_commit_count, "author_name ", author_name)
                    
                    csv_results_for_author_commit.append( [base, head, author_name, authors_commit_count] )
                
                csv_results.append([base, head, addition_deletion_list[1], addition_deletion_list[4], addition_deletion_list[6], total_change, int(commit_no), int(contributor_no) ])
                print("===============================================")
            except:
                print("Exception occured")
                continue   
        print("done")        
        
    @staticmethod
    def _authenticate_to_github(access_token: str, password: str, user_name: str):
        if access_token is None:
            cred: Github = Github(user_name, password)
        else:
            cred: Github = Github(access_token)
        return cred



In [ ]:
if __name__ == "__main__":
    token = "Provide your github access token"
    github = GithubStatisticsGeneration(access_token= token)
    
    owner_project = owner + "/" + project
    github.get_tags(owner_project)
    
    result_df = pd.DataFrame(data=csv_results, columns = labels )
    result_df.to_csv ('../' + project + '.csv', sep=",", index=None)
    print(result_df.head(3))
    print("========================================================================\n")
    result_df_for_author_commit = pd.DataFrame(data=csv_results_for_author_commit, columns = labels_for_author_commit )
    result_df_for_author_commit.to_csv ('../' + project + '_authors_individual_contribution.csv', sep=",", index=None)
    print(result_df_for_author_commit.head(3))